# Notebook 5 - Computer Vision Grundlagen


## 1) Lernziele
- Du liest Bilddaten als Matrix mit Intensitaetswerten.
- Du verstehst Faltung als lokale Rechenregel auf Pixelnachbarschaften.
- Du vergleichst Filterwirkungen (blur, sharpen, edge) interaktiv.
- Du leitest einfache Features (Kanten, Ecken) und Segmentierungsschritte her.
- Du interpretierst Grenzen durch Rauschen, Thresholds und Parameterwahl.


## 2) Datensatz Einblick: Bildmatrix, Pixelwerte, Ausschnitt als Heatmap


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
from scipy.signal import convolve2d


def load_image():
    # Kein Download: lokale Demo-Bilder aus skimage.data.
    try:
        from skimage import data
        img = data.camera().astype(np.float32)
        source = 'skimage.data.camera'
    except Exception:
        # Robuster Fallback: kleines generiertes Bild.
        h, w = 256, 256
        y = np.linspace(0, 255, h)[:, None]
        x = np.linspace(0, 255, w)[None, :]
        img = 0.65 * x + 0.35 * y
        img[80:180, 90:190] = 255 - img[80:180, 90:190]
        img = img.astype(np.float32)
        source = 'generated_fallback'
    return img, source


IMG, IMG_SOURCE = load_image()
print('Bildquelle:', IMG_SOURCE)
print('shape:', IMG.shape)
print('dtype:', IMG.dtype)
print('min/max:', float(IMG.min()), float(IMG.max()))

patch20 = IMG[90:110, 90:110]
plt.figure(figsize=(4.2, 3.8))
im = plt.imshow(patch20, cmap='viridis')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('20x20 Ausschnitt als Heatmap')
plt.tight_layout()
plt.show()


## 3) Warm-up Spielzelle: Filter-Spielplatz (blur, sharpen, edge)
- Regler: `filter`, `kernel_strength`, `threshold`, `noise`.
- Ausgabe: Original, gefiltertes Bild, optionale Kantenmaske.


In [ ]:
def normalize01(img):
    m, M = float(img.min()), float(img.max())
    if M - m < 1e-8:
        return np.zeros_like(img)
    return (img - m) / (M - m)


def make_kernel(filter_name, strength):
    s = float(strength)
    if filter_name == 'blur':
        k_blur = np.ones((3, 3), dtype=np.float32) / 9.0
        k_id = np.zeros((3, 3), dtype=np.float32)
        k_id[1, 1] = 1.0
        alpha = min(max(s / 3.0, 0.0), 1.0)
        return (1 - alpha) * k_id + alpha * k_blur
    if filter_name == 'sharpen':
        return np.array([
            [0, -1, 0],
            [-1, 1 + 4 * s, -1],
            [0, -1, 0],
        ], dtype=np.float32)
    return s * np.array([
        [-1, -1, -1],
        [-1,  8, -1],
        [-1, -1, -1],
    ], dtype=np.float32)


def apply_filter(img01, kernel):
    out = convolve2d(img01, kernel, mode='same', boundary='symm')
    return normalize01(out)


filter_dd = widgets.Dropdown(options=['blur', 'sharpen', 'edge'], value='blur', description='filter')
strength_sl = widgets.FloatSlider(value=1.0, min=0.2, max=3.0, step=0.1, description='kernel_strength', continuous_update=False)
threshold_sl = widgets.FloatSlider(value=0.35, min=0.0, max=1.0, step=0.05, description='threshold', continuous_update=False)
noise_sl = widgets.FloatSlider(value=0.0, min=0.0, max=0.3, step=0.02, description='noise', continuous_update=False)
show_mask_cb = widgets.Checkbox(value=True, description='show_edge_mask')
run_btn = widgets.Button(description='Apply', button_style='info')
out = widgets.Output()


def on_apply(_):
    with out:
        out.clear_output()

        rng = np.random.default_rng(42)
        img01 = normalize01(IMG)
        noisy = np.clip(img01 + rng.normal(0.0, noise_sl.value, img01.shape), 0.0, 1.0)

        k = make_kernel(filter_dd.value, strength_sl.value)
        filt = apply_filter(noisy, k)
        edge_mask = (filt >= threshold_sl.value).astype(np.float32)

        ncols = 3 if show_mask_cb.value else 2
        fig, axes = plt.subplots(1, ncols, figsize=(5.2 * ncols, 4.0))
        if ncols == 2:
            axes = np.array(axes)

        axes[0].imshow(noisy, cmap='gray', vmin=0, vmax=1)
        axes[0].set_title('Original + Noise')
        axes[0].axis('off')

        axes[1].imshow(filt, cmap='gray', vmin=0, vmax=1)
        axes[1].set_title(f'Filtered: {filter_dd.value}')
        axes[1].axis('off')

        if show_mask_cb.value:
            axes[2].imshow(edge_mask, cmap='gray', vmin=0, vmax=1)
            axes[2].set_title(f'Edge Mask (thr={threshold_sl.value:.2f})')
            axes[2].axis('off')

        plt.tight_layout()
        plt.show()

        print('Kernel:')
        print(np.round(k, 3))


run_btn.on_click(on_apply)

display(widgets.VBox([
    filter_dd,
    strength_sl,
    threshold_sl,
    noise_sl,
    show_mask_cb,
    run_btn,
    out,
]))


## 4) Convolution Zahlenbeispiel: 3x3 Patch und Kernel als Tabellen + Rechnung


In [ ]:
import pandas as pd

patch = np.array([
    [0.20, 0.30, 0.20],
    [0.10, 0.90, 0.20],
    [0.00, 0.20, 0.10],
], dtype=np.float32)

kernel = np.array([
    [-1.0, -1.0, -1.0],
    [-1.0,  8.0, -1.0],
    [-1.0, -1.0, -1.0],
], dtype=np.float32)

product = patch * kernel
output_pixel = float(product.sum())

# Tabellenansicht fuer Rechenschritte
patch_df = pd.DataFrame(patch)
kernel_df = pd.DataFrame(kernel)
prod_df = pd.DataFrame(np.round(product, 3))

print('Patch (3x3):')
display(patch_df)
print('Kernel (3x3):')
display(kernel_df)
print('Elementweise Multiplikation:')
display(prod_df)
print('Output-Pixel = Summe aller 9 Produkte =', round(output_pixel, 4))

plt.figure(figsize=(3.2, 3.0))
im = plt.imshow(kernel, cmap='coolwarm')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('Kernel Heatmap')
plt.tight_layout()
plt.show()


## 5) Feature Detection Block: Edges/Corners und Qualitaetskriterien
### Qualitaetskriterien
- Robustness: Features bleiben unter Rauschen und Beleuchtungsunterschieden stabil.
- Repeatability: dieselben Punkte/Kanten werden in aehnlichen Bildern wiedergefunden.
- Distinctiveness: Features sind eindeutig genug fuer Matching.
- Efficiency: Berechnung bleibt fuer grosse Bilder praktikabel.
- Localization: erkannte Positionen sind praezise.


In [ ]:
# Leichtgewichtige Feature-Detektion: Sobel-Kanten + einfacher Harris-aehnlicher Corner-Score.
img01 = normalize01(IMG)

sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
sobel_y = sobel_x.T
Ix = convolve2d(img01, sobel_x, mode='same', boundary='symm')
Iy = convolve2d(img01, sobel_y, mode='same', boundary='symm')
edge_mag = np.sqrt(Ix**2 + Iy**2)
edge_mag = normalize01(edge_mag)

# Harris-aehnlicher Score mit kleiner Fensterglaettung.
win = np.ones((3, 3), dtype=np.float32) / 9.0
Ixx = convolve2d(Ix * Ix, win, mode='same', boundary='symm')
Iyy = convolve2d(Iy * Iy, win, mode='same', boundary='symm')
Ixy = convolve2d(Ix * Iy, win, mode='same', boundary='symm')

k = 0.04
harris = (Ixx * Iyy - Ixy * Ixy) - k * (Ixx + Iyy) ** 2
harris = normalize01(harris)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].imshow(img01, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(edge_mag, cmap='magma', vmin=0, vmax=1)
axes[1].set_title('Edge Magnitude (Sobel)')
axes[1].axis('off')

axes[2].imshow(harris, cmap='inferno', vmin=0, vmax=1)
axes[2].set_title('Corner Score (Harris-like)')
axes[2].axis('off')

plt.tight_layout()
plt.show()


## 6) Calibration und Segmentation als Konzept mit Mini-Demo
- Calibration (Konzept): Pixelwerte/Modelle auf verlässliche Skalen und reale Bedingungen abstimmen.
- Segmentierung (Mini-Demo): Trenne Vordergrund/Hintergrund via einfachem Threshold.
- Overlay hilft, Segmentierung direkt im Kontext des Originalbilds zu beurteilen.


In [ ]:
img01 = normalize01(IMG)
threshold = 0.45
mask = (img01 >= threshold).astype(np.float32)

# Overlay: Original in Graustufen, Maske rot mit Alpha.
overlay_rgb = np.dstack([img01, img01, img01])
red = np.zeros_like(overlay_rgb)
red[..., 0] = 1.0
alpha = 0.35
overlay = np.where(mask[..., None] > 0, (1 - alpha) * overlay_rgb + alpha * red, overlay_rgb)

overlay = np.clip(overlay, 0, 1)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].imshow(img01, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(mask, cmap='gray', vmin=0, vmax=1)
axes[1].set_title(f'Binary Mask (thr={threshold:.2f})')
axes[1].axis('off')

axes[2].imshow(overlay)
axes[2].set_title('Overlay (Alpha Blend)')
axes[2].axis('off')

plt.tight_layout()
plt.show()


## 7) Aufsteigende Erweiterungen: noise vs smoothing, thresholding, simple segmentation
- Noise vs Smoothing: mehr Rauschen reduziert Kantenstabilitaet, Blur kann stabilisieren.
- Thresholding: kleine Thresholds liefern viele Pixel, grosse Thresholds nur starke Strukturen.
- Simple Segmentation: globale Schwellwerte sind schnell, aber empfindlich gegen Beleuchtungswechsel.


In [ ]:
# Kleine Vergleichsdemo: noise vs smoothing + threshold sweep als kompakte Tabellenwerte.
img01 = normalize01(IMG)
rng = np.random.default_rng(7)

noise_levels = [0.00, 0.08, 0.16]
rows_noise = []
for nl in noise_levels:
    noisy = np.clip(img01 + rng.normal(0.0, nl, img01.shape), 0.0, 1.0)
    k_blur = np.ones((3, 3), dtype=np.float32) / 9.0
    smooth = convolve2d(noisy, k_blur, mode='same', boundary='symm')

    ex = convolve2d(smooth, np.array([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=np.float32), mode='same', boundary='symm')
    ey = convolve2d(smooth, np.array([[-1,-2,-1],[0,0,0],[1,2,1]], dtype=np.float32), mode='same', boundary='symm')
    em = normalize01(np.sqrt(ex**2 + ey**2))

    rows_noise.append({'noise': nl, 'mean_edge_after_smoothing': float(em.mean())})

noise_df = pd.DataFrame(rows_noise)
display(noise_df)

thresholds = [0.25, 0.40, 0.55, 0.70]
rows_thr = []
for t in thresholds:
    mask = (img01 >= t).astype(np.float32)
    rows_thr.append({'threshold': t, 'foreground_ratio': float(mask.mean())})

thr_df = pd.DataFrame(rows_thr)
display(thr_df)


## 8) Mini Leitfaden (7 bis 10 Minuten)
- Minute 0-1: Datensatz-Einblick lesen (shape, min/max, Heatmap-Ausschnitt).
- Minute 1-3: Warm-up mit `blur` starten und `noise` schrittweise erhoehen.
- Minute 3-5: `edge` und `threshold` kombinieren, Maskenwirkung vergleichen.
- Minute 5-7: Zahlenbeispiel der Convolution durchgehen und Summe nachvollziehen.
- Minute 7-9: Feature-Detection-Block betrachten (Kanten vs Corner-Score).
- Minute 9-10: Segmentierungs-Overlay interpretieren und Grenzen notieren.


## Mini Uebungen
1. Finde eine Filtereinstellung, bei der Kanten deutlich sind, aber Rauschen gering bleibt.
2. Aendere den 3x3-Kernel im Zahlenbeispiel und berechne den neuen Output-Pixel.
3. Waehle zwei Thresholds, die sich im foreground_ratio stark unterscheiden, und erklaere warum.
4. Vergleiche Corner-Score und Edge-Magnitude an derselben Bildstelle.
5. Beschreibe in drei Stichpunkten, wann globale Threshold-Segmentierung scheitert.
